# Publication figures 
This script calls all python scripts in the same order as appearing in the manuscript which were created using the bioinformatics pipeline.

## Table of contents
* <a href=#DD_prep>0 Preparations</a>
   * <a href=#importing>0.1 Importing packages</a>
   * <a href=#functions>0.2 Create functions</a>
   * <a href=#initparameters>0.3 Init Parameters</a>
   * <a href=#Inf_stdata>0.4 Information about spatial data</a>
* <a href=#read-data>1 Load data</a>
* <a href=#MainFigures>2 Main Figures</a>
* <a href=#SuppFigures>3 Supplemental Figures</a>
* <a href=#SuppTables>4 Supplemental Tables</a>

# 0. Preparations

## 0.1 Importing packages

In [2]:
import sys  
sys.path.insert(0, '/Users/christina.hillig/PycharmProjects/ST_Immune_publication/Publication_analysis/python_scripts')

In [1]:
# Import Main figures scripts
from python_scripts.analysis.figure_1 import UMAP_bulkRNAseq
from python_scripts.analysis.figure_2 import Fig2A__ST_sections, Fig2B__cytokine_tissuelayer_expression, Fig2C__SC_cytokine_expression, Fig2DF__Bulk_cytokine_expression, Fig2NQ__ST_cytokine_counts
from python_scripts.analysis.figure_3 import Fig3A__ST_UMAP_diseases, Fig3B__ST_UMAP_IL17A_tissuelayers, Fig3CD__ST_Volcanoplot_cytokine
from python_scripts.analysis.figure_4 import Fig4A__SC_UMAP_clusters, Fig4B__SC_UMAP_IL17A, Fig4C__SC_Volcano_plot
from python_scripts.analysis.figure_5 import Fig5AC__ST_pseudobulk_aggregation_Correlation, Fig4FH__Weighted_Correlation
# Creates Figure 5G-I
from python_scripts.spatial_correlation import main as csdcc

ModuleNotFoundError: No module named 'python_scripts'

In [ ]:
# Import Supplemental figures scripts
from python_scripts.analysis.supplfigure_1 import SuppFig1DE__cytokine_counts_skinlayers, \
    SuppFig1C__Lesion_NonLesion_Mean_expression_cytokines_others
from python_scripts.analysis.supplfigure_3 import SuppFig2ABC__ST_UMAP
from python_scripts.analysis.supplfigure_4 import SupplFig3AF__spot_celltypes
from python_scripts.analysis.supplfigure_5 import SuppFig4AC__ST_Volcano_Boxplot
from python_scripts.analysis.supplfigure_6 import SuppFig5A__SC_UMAP_IFNG, SuppFig5B__SC_Volcano_IFNG
from python_scripts.analysis.supplfigure_7 import SuppFig7AC__ST_Common_genes_cytokinepositive_condition, \
    SuppFig7H__cytokines_vs_responders
from python_scripts.analysis.supplfigure_8 import SuppFig8A__ST_pseudobulk_Correlation_permutedresponders, \
    SuppFig8AC__refined_responds

In [ ]:
# Import supplemental table script
from python_scripts.analysis.suppltable import SuppTab2__overview_counts

In [ ]:
# Import utils
from python_scripts.utils import gene_lists

In [ ]:
# Import Modules
import os
from datetime import date
import scanpy as sc
import numpy as np
import pandas as pd

## 0.2 Create functions

In [ ]:
def get_deg_files(input_dir, cytokine):
    design_function_spatial = 'cdr_project_patient_annotation_cyto'
    design_function_singlecell = 'cdr_annotation_cyto'
    dataset = "Whole_T_cell_matrix"

    # load df's using pandas
    # Spatial DEGs paths
    comparisons = "_".join([cytokine, 'vs_Others'])
    st_path = os.path.join(
        input_dir, "dge_analysis",
        "".join(['2022-04-08_spatial_', design_function_spatial, os.path.sep, 'spatial',
                 os.path.sep, dataset, "__", design_function_spatial, os.path.sep, cytokine, os.path.sep,
                 comparisons]))
    st_file = os.path.join(
        input_dir, "dge_analysis", st_path,
        "".join([dataset, "_", comparisons, "_glmGamPoi_DGE_all_genes.csv"]))
    
    # Single cell DEGs paths
    sc_path = os.path.join(
        input_dir, "dge_analysis", '2021-02-01_single_cell__', design_function_singlecell, 'single_cell',
        "".join([dataset, "__", design_function_singlecell]), cytokine, comparisons)
    sc_file = os.path.join(
        input_dir, "dge_analysis", sc_path,
        "".join(["single_cell_", comparisons, "_glmGamPoi_DGE_all_genes.csv"]))

    return st_path, st_file, sc_path, sc_file

In [ ]:
def load_spatial_data(adata_folder):
    date_st_unpp = '2022-04-08' 
    date_st_pp = '2022-04-08'  
    
    # load unpreprocessed data
    unpp_adata = sc.read(os.path.join(
        st_adata_folder, date_st_unpp, "Spatial Transcriptomics_unpp_cleaned_LPADPso.h5"))

    # load preprocessed data
    pp_adata = sc.read(os.path.join(st_adata_folder, date_st_pp, 'st_QC_normed_BC_project_PsoADLP.h5'))
    pp_adata = check_tissuelayer_annotation(st_adata=pp_st_adata)
    
    return unpp_adata, pp_adata

    
def load_singlecell_data(adata_folder):
    sc_adata_folder = '/Users/christina.hillig/Documents/Projects/annData_objects'
    unpp_adata = sc.read(os.path.join(sc_adata_folder, '2020-11-30', 'sc_adata_unpp.h5'))
    pp_adata = sc.read(os.path.join(sc_adata_folder, '2020-12-04_SC_Data_QC_clustered.h5'))
    
    return unpp_adata, pp_adata


    
def load_bulk_data(data_folder):
    # --> Bulk RNAseq data
    input_rnaseq = os.path.join(input_folder, "bulk_RNAseq")
    # - Read bulk-RNAseq count matrix
    data = pd.read_csv(os.path.join(input_rnaseq, "bulkRNA_countMat.txt"), sep='\t')
    # - Read in metaData
    metadata = pd.read_excel(os.path.join(input_rnaseq, "bulkRNA_metaData.xlsx"))
    
    return data, metadata

## 0.3 Init Parameters


In [ ]:
# Save directory
today = date.today()
project_folder = os.path.join("..", "..")
save_folder = os.path.join(project_folder, "output")
os.makedirs(save_folder, exist_ok=True)

# Input directory
input_path = os.path.join(project_folder, "input")

# Data
# --> Spatial Transcriptomics data
st_adata_folder = os.path.join(project_folder, "adata_storage")
# --> Single cell RNAseq data
sc_adata_folder = '/Users/christina.hillig/Documents/Projects/annData_objects'
unpp_sc_adata = sc.read(os.path.join(sc_adata_folder, '2020-11-30', 'sc_adata_unpp.h5'))
pp_sc_adata = sc.read(os.path.join(sc_adata_folder, '2020-12-04_SC_Data_QC_clustered.h5'))
# --> Bulk RNAseq data
bulk_data_folder = input_folder

In [ ]:
# Files
# DEGs
st_path_il17a, st_file_il17a, sc_path_il17a, sc_file_il17a = get_deg_files(input_dir=input_folder, cytokine='IL17A')
st_path_ifng, st_file_ifng, sc_path_ifng, sc_file_ifng = get_deg_files(input_dir=input_folder, cytokine='IFNG')
input_st_dgeanalysis = os.path.join(
    input_folder, 'dge_analysis', '2022-04-08_spatial__cdr_project_patient_annotation_cyto')

# Spatial Correlation
radius = list(np.arange(0, 5, 1))
corr_method = 'spearman'
t_cell_cytocines, cyto_resps_list, cytokine_responders = gene_lists.get_publication_cyto_resps()

## Information about the spatial data
- sample: technical sample
- project: from sequencing batch the samples originate
- capture area: a capture area on the Visium object slide
- object slide: Visium object slide
- specimen: a tissue slice
- SAMPLE: clinical patient IDs
- patients: patient IDs 1-40

<br><br><br>

# 1. Load data

In [ ]:
# Spatial 
unpp_st_adata, pp_st_adata = load_spatial_data(adata_folder=st_adata_folder)

In [ ]:
# Single cell
unpp_sc_adata, pp_sc_adata = load_singlecell_data(adata_folder=sc_adata_folder)

In [ ]:
# Bulk 
bulk_data, meta_data = load_bulk_data(data_folder=bulk_data_folder)

<br><br><br>

# 2. Main Figures

## 2.1 Figure 1

In [ ]:
save_folder_fig1 = os.path.join(save_folder, 'Fig1', str(date.today()))
os.makedirs(save_folder_fig1, exist_ok=True)

UMAP_bulkRNAseq.main(save_folder=save_folder_fig1, bulk_rnaseq=bulk_data, metadata=meta_data)

## 2.2 Figure 2

In [ ]:
save_folder_fig2 = os.path.join(save_folder, 'Fig2', str(date.today()))
os.makedirs(save_folder_fig2, exist_ok=True)
    
Fig2A__ST_sections.main(save_folder=save_folder_fig2, adata=unpp_st_adata)
Fig2B__cytokine_tissuelayer_expression.main(save_folder_fig2=save_folder, adata=unpp_st_adata)
Fig2C__SC_cytokine_expression.main(save_folder=save_folder_fig2, adata=unpp_sc_adata)
Fig2DF__Bulk_cytokine_expression.main(save_folder=save_folder_fig2, bulk_rnaseq=bulk_data, metadata=meta_data)
# Fig2N-P: Cytokine counts of IFNG, IL17A, IL13 resolved by disease
Fig2NQ__ST_cytokine_counts.get_per_diagnosis_cytokine_counts(unpp_st_adata, save_folder_fig2, biopsy_type='LESIONAL')
Fig2NQ__ST_cytokine_counts.get_per_diagnosis_cytokine_counts(unpp_st_adata, save_folder_fig2, biopsy_type='NON LESIONAL')
# Fig2Q: Total number of cytokine counts of IFNG, IL17A, IL13 resolved by disease
Fig2NQ__ST_cytokine_counts.get_cytokinecounts_per_diagnosis(unpp_st_adata, save_folder_fig2, biopsy_type='LESIONAL')
Fig2NQ__ST_cytokine_counts.get_cytokinecounts_per_diagnosis(unpp_st_adata, save_folder_fig2, biopsy_type='NON LESIONAL')

## 2.3 Figure 3

In [ ]:
save_folder_fig3 = os.path.join(save_folder, 'Fig3', str(date.today()))
os.makedirs(save_folder_fig3, exist_ok=True)

Fig3A__ST_UMAP_diseases.main(save_folder=save_folder_fig3, spatial_adata=pp_st_adata)
Fig3B__ST_UMAP_IL17A_tissuelayers.main(save_folder=save_folder_fig3, spatial_adata=pp_st_adata)
# Figure 3CD: Plots Volcano plot (C) and Violinplots (D)
# df_keys contains column names of DEGs dataframe. 
# First string should be the name of the Log2FC column, 
# Second of either p-value or p-adjusted value,
# Third should be the name of the HNGO gene names
Fig3CD__ST_Volcanoplot_cytokine.main(
    adata=pp_st_adata, save_folder=save_folder_fig3, df_keys=['log2fc', 'padj', 'gene_symbol'],
    log=False, dge_results_folder=input_st_dgeanalysis)

## 2.4 Figure 4

In [ ]:
save_folder_fig4 = os.path.join(save_folder, 'Fig4', str(date.today()))
os.makedirs(save_folder_fig4, exist_ok=True)

Fig4A__SC_UMAP_clusters.main(save_folder=save_folder, pp_adata=pp_sc_adata, cluster_algorithm='leiden')
Fig4B__SC_UMAP_IL17A.main(save_folder=save_folder, adata=pp_sc_adata)
# Figure 4C
# df_keys contains column names of DEGs dataframe. 
# First string should be the name of the Log2FC column, 
# Second of either p-value or p-adjusted value,
# Third should be the name of the HNGO gene names
Fig4C__SC_Volcano_plot.main(
    dataset_type='SC', save_folder=save_folder_fig4, df_keys=['log2fc', 'padj', 'gene_symbol'],
    log=False, dge_results_folder=sc_path_il17a)

## 2.5 Figure 5

In [ ]:
save_folder = '/Volumes/CH__data/ST_immune_publication/Revision/Fig5'
save_folder_fig5ac = os.path.join(save_folder, 'Bulk_Weighted_Spearman', 'Unweighted_fit', str(date.today()))
os.makedirs(save_folder_fig5ac, exist_ok=True)
# Run Weighted correlation by cytokine+ spots in epidermis for pseudo-bulk approach
fig5__dict_weighted_transcripts_corr, df_bulk = Fig5AC__ST_pseudobulk_aggregation_Correlation.main(
        save_folder=save_folder_fig5ac, adata=unpp_st_adata, corr_method=corr_method)

# Create figure 5D-G - if get_plots is set to True, otherwise it will only create the plots for 5E-G
save_folder_fig5eg = os.path.join(save_folder, 'Weighted_Spearman_unppadata', 'Unweighted_fit', str(date.today()))
os.makedirs(save_folder_fig5eg, exist_ok=True)
counts_dict, df_stats_responders_in_vs_outlesion_sdc, df_stats_cytokines_responders_in_sdc = csdcc.main(
    save_folder=save_folder_fig5eg, adata=unpp_st_adata, corr_method=corr_method, get_plots=False,
    find_responders=False, radius=radius,
    cond_genes=t_cell_cytocines, genes_resps=cytokine_responders)

<br><br><br>

# 3. Supplemental Figures

## Figure S1

In [ ]:
save_folder_sfig1 = os.path.join(save_folder, 'SFig1')
os.makedirs(save_folder_sfig1, exist_ok=True)

# Creates also supplemental figure 1A
Fig2B__cytokine_tissuelayer_expression.main(save_folder_fig2=save_folder_sfig1, adata=unpp_st_adata)
SuppFig1DE__cytokine_counts_skinlayers.main(save_folder=save_folder_sfig1, adata=unpp_st_adata)
df_stats_mean_cytokines_others_l_vs_nl = SuppFig1C__Lesion_NonLesion_Mean_expression_cytokines_others.main(
    save_folder=save_folder_sfig1, pp_st_adata=pp_st_adata)

## Figure S3

In [ ]:
""" SFig 3 """
save_folder_sfig3 = os.path.join(save_folder, 'SFig3')
os.makedirs(save_folder_sfig3, exist_ok=True)

SuppFig2ABC__ST_UMAP.main(
    save_folder=save_folder_sfig3, spatial_adata=pp_st_adata, spatial_cluster_label='tissue_layer')

## Figure S4
Deconvolution of spots using _Tangram_ and a public single cell dataset as reference. <br>
Please first run the jupyter-notebooks to deconvolute the spots and then run this function.

In [ ]:
""" SFig 4 """  
save_folder_sfig4 = os.path.join(save_folder, 'SFig4')
os.makedirs(save_folder_sfig4, exist_ok=True)

# Call jupyternotebooks before running this line
SupplFig3AF__spot_celltypes.main(save_folder=save_folder_sfig4)

## Figure S5

In [ ]:
save_folder_sfig5 = os.path.join(save_folder, 'SFig5')
os.makedirs(save_folder_sfig5, exist_ok=True)

SuppFig4AC__ST_Volcano_Boxplot.main(
    adata=pp_st_adata, save_folder=save_folder_sfig5, df_keys=['log2fc', 'padj', 'gene_symbol'],
    log=False, dge_results_folder=st_path_ifng)

## Figure S6

In [ ]:
save_folder_sfig6 = os.path.join(save_folder, 'SFig6')
os.makedirs(save_folder_sfig6, exist_ok=True)

SuppFig5A__SC_UMAP_IFNG.main(save_folder=save_folder_sfig6, adata=pp_sc_adata)
SuppFig5B__SC_Volcano_IFNG.main(
    dataset_type='SC', save_folder=save_folder_sfig6, df_keys=['log2fc', 'padj', 'gene_symbol'],
    log=False, dge_results_folder=sc_path_ifng)

## Figure S7

In [ ]:
save_folder_sfig7 = os.path.join(save_folder, 'SFig7')
os.makedirs(save_folder_sfig7, exist_ok=True)

SuppFig7AC__ST_Common_genes_cytokinepositive_condition.main(
    input_path=input_st_dgeanalysis, save_folder=save_folder_sfig7)
df_stats_lesion_mean_persample_perspecimen_cytokines_responders = SuppFig7H__cytokines_vs_responders.main(
    save_folder=save_folder_sfig7, pp_st_adata=pp_st_adata)

## Figure S8

In [ ]:
save_folder_sfig8 = os.path.join(save_folder, 'Figure_S8', str(date.today()))
os.makedirs(save_folder_sfig8, exist_ok=True)

# 1. Run with preprocessed (QC, normed, bc) data density clustering
_, _, _ = csdcc.main(save_folder=save_folder_sfig8, adata=pp_st_adata, corr_method=corr_method, get_plots=False,
                     find_responders=True, radius=list(np.arange(0, 5, 1)), cond_genes=t_cell_cytocines,
                     genes_resps=cytokine_responders)
# 2. Identify DEGs (python) and create Volcano plots (R script)
# 3. Identify new responder genes using DEGs from radii 1-5 and rerun density clustering
SuppFig8AC__refined_responds.main(unpp_st_adata=unpp_st_adata)

<br><br><br>

# 4. Supplemental Tables

In [ ]:
# Supplemental Table 2
SuppTab2__overview_counts.create_supptable_1(unpp_st_adata=unpp_st_adata, save_folder=save_folder)

In [ ]:
# Supplemental Table only for verification of numbers in manuscript
writer = pd.ExcelWriter(os.path.join(save_folder, 'Supplemental_Table.xlsx'), engine='xlsxwriter')
df_stats_responders_in_vs_outlesion_sdc.to_excel(writer, sheet_name='Abstract')
df_stats_mean_cytokines_others_l_vs_nl.to_excel(writer, sheet_name='FigS1C')
df_stats_lesion_mean_persample_perspecimen_cytokines_responders.to_excel(writer, sheet_name='FigS7')
writer.save()
writer.close()